# Automatic Differentiation

```
x = 5
y = x^2


(dy)/(dx) = 2x = 2(5) = 10
```

In [1]:
import torch

In [2]:
x = torch.tensor(5.0)
x

tensor(5.)

In [3]:
x.requires_grad_() # contagiously track gradients through forward pass

tensor(5., requires_grad=True)

In [4]:
y = x**2

In [5]:
y.backward() # use autodiff

In [6]:
x.grad

tensor(10.)

Repeat but with TensorFlow